## GPT3.5 Function Calling am Beispiel von einer Flugabfrage mit mehreren Funktionen

In [2]:
# --------------------------------------------------------------
# Import Modules
# --------------------------------------------------------------

import os
import json
import openai
from datetime import datetime, timedelta
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage



In [3]:

# --------------------------------------------------------------
# Load OpenAI API Token From the .env File
# --------------------------------------------------------------

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")



### Ein einfaches Function Calling Beispiel

Julia ist die persönliche Assistenz von Noah <br>
Niklas ist die persönliche Assistenz von Kaan

In [41]:
# Hier nutzen wir 3 Funktionen
function_description_Niklas = [
    {
        "name": "askJulia",
        "description": "Julia ist die persönliche Assistenz von Noah. Alle Informationen auf die du selber keinen Zugriff hast sollst du Julia Fragen.",
        "parameters": {
            "type": "object",
            "properties": {
                "prompt": {
                    "type": "string",
                    "description": "Deine Frage an Julia. Bedenke, dass Julia ein Sprachmodell von OpenAi ist. Schreibe also einen guten Prompt an Sie",
                },
            },
            "required": ["prompt"],
        },
    },
    {
        "name": "getKalenderKaan",
        "description": "Kriege alle Informationen ausschließlich von Kaans Kalender",
        "parameters": {
            "type": "object",
            "properties": {
                "datum": {
                    "type": "string",
                    "description": "Das Datum an dem geschaut werden soll, welche Termine es gibt. Das Datum soll im Format DD.MM.YYYY sein, also der 18.Dezember 2023 ist 18.12.2023",
                },
            },
            "required": ["datum"],
        },
    },
]
function_description_Julia = [
    {
        "name": "askNiklas",
        "description": "Niklas ist die persönliche Assistenz von Kaan. Wenn du Informationen über Kaan brauchst dann frag Niklas",
        "parameters": {
            "type": "object",
            "properties": {
                "prompt": {
                    "type": "string",
                    "description": "Deine Frage an Niklas über Kaan. Bedenke, dass Niklas ein Sprachmodell von OpenAi ist. Schreibe also einen guten Prompt an Sie ",
                },
            },
            "required": ["prompt"],
        },
    },
    {
        "name": "getKalenderNoah",
        "description": "Kriege Informationen über Noahs Kalender. RUFE DIESE FUNKTION NUR FÜR NOAHS KALENDAR AUF",
        "parameters": {
            "type": "object",
            "properties": {
                "datum": {
                    "type": "string",
                    "description": "Das Datum an dem geschaut werden soll, ob Noah noch einen Termin frei hat. Das Datum soll im Format DD.MM.YYYY sein, also der 18.Dezember 2023 ist 18.12.2023",
                },
            },
            "required": ["datum"],
        },
    },
]

def askJulia(prompt):
    llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)
    output = llm.predict_messages(
        [HumanMessage(content=prompt)], functions=function_description_Julia
    )

    return output

def getKalenderNoah(datum):
    output = ""
    if datum == "30.12.2023":
        output = {
            "datum": "30.12.2023",
            "termine": "Meeting beim Startplatz"
        }
    else:
        output = {
            "datum": datum,
            "termine": ""
        }
    return output

def askNiklas(prompt):
    llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)
    output = llm.predict_messages(
        [HumanMessage(content=prompt)], functions=function_description_Niklas
    )

    return output


def getKalenderKaan(datum):
    output = ""
    if datum == "01.12.2023":
        output = {
            "datum": "01.12.2023",
            "termine": "Meeting beim Startplatz"
        }
    else:
        output = {
            "datum": datum,
            "termine": ""
        }
    return output


In [39]:
user_prompt = "Hey Julia, hat Kaan am 1.12.2023 einen Termin?"
first_response_from_julia = askJulia(user_prompt)


print("Aufgerufene Funktion: " + first_response_from_julia.additional_kwargs["function_call"]["name"])
print(first_response_from_julia)
chosen_function = eval(first_response_from_julia.additional_kwargs["function_call"]["name"])
params = json.loads(first_response_from_julia.additional_kwargs.get("function_call").get("arguments"))


Aufgerufene Funktion: askNiklas
content='' additional_kwargs={'function_call': {'name': 'askNiklas', 'arguments': '{\n  "prompt": "Hat Kaan am 1.12.2023 einen Termin?"\n}'}} example=False


In [43]:
response_niklas_first = chosen_function(**params)

print(response_niklas_first)
chosen_function = eval(response_niklas_first.additional_kwargs["function_call"]["name"])

content='' additional_kwargs={'function_call': {'name': 'getKalenderKaan', 'arguments': '{\n  "datum": "01.12.2023"\n}'}} example=False


In [44]:
params = json.loads(response_niklas_first.additional_kwargs.get("function_call").get("arguments"))
function_output = chosen_function(**params)
print(function_output)

{'datum': '01.12.2023', 'termine': 'Meeting beim Startplatz'}
